Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

/home/florian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = '../lesson_2/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Regularization for logit

In [12]:
batch_size = 128
beta_l2_logit = 0.1


graph_logit = tf.Graph()
with graph_logit.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                logits=logits))
          + 0.5 * tf.nn.l2_loss(weights) * beta_l2_logit
          + 0.5 * tf.nn.l2_loss(biases) * beta_l2_logit)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,
                                             weights)
                                   + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,
                                            weights)
                                  + biases)

In [13]:
num_steps = 3001

with tf.Session(graph=graph_logit) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 171.660370
Minibatch accuracy: 12.5%
Validation accuracy: 12.7%
Minibatch loss at step 500: 0.959273
Minibatch accuracy: 79.7%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 1.064678
Minibatch accuracy: 75.8%
Validation accuracy: 77.9%
Minibatch loss at step 1500: 1.035628
Minibatch accuracy: 71.9%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 0.914261
Minibatch accuracy: 78.9%
Validation accuracy: 76.7%
Minibatch loss at step 2500: 0.783527
Minibatch accuracy: 85.2%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 0.959366
Minibatch accuracy: 74.2%
Validation accuracy: 78.9%
Test accuracy: 85.5%


### Regularization for 1-layer NN

In [6]:
num_hidden_layers = 1024
beta_l2_1_layer_sgd = 0.1


batch_size = 128

graph_1_layer_sgd = tf.Graph()
with graph_1_layer_sgd.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size,
                                             num_hidden_layers]))
  biases = tf.Variable(tf.zeros([num_hidden_layers]))
  preactivations_hidden = tf.add(tf.matmul(tf_train_dataset, weights), biases)

  # Hidden layer.
  activations_hidden = tf.nn.relu(preactivations_hidden)
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden_layers, num_labels]))
  bias_hidden = tf.Variable(tf.random_normal([num_labels]))
  logit_hidden = tf.add(tf.matmul(activations_hidden, weights_hidden), bias_hidden)
    
    
  # Training computation.
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                logits=logit_hidden))
          + 0.5 * tf.nn.l2_loss(weights) * beta_l2_1_layer_sgd
          + 0.5 * tf.nn.l2_loss(biases) * beta_l2_1_layer_sgd
          + 0.5 * tf.nn.l2_loss(weights_hidden) * beta_l2_1_layer_sgd
          + 0.5 * tf.nn.l2_loss(bias_hidden) * beta_l2_1_layer_sgd
         )


  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logit_hidden)
  valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset,
                                                                                   weights),
                                                                         biases)),
                                                    weights_hidden),
                                          bias_hidden))
  test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset,
                                                                                  weights),
                                                                         biases)),
                                                    weights_hidden),
                                          bias_hidden))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [30]:
batch_size = 128
num_steps = int(train_labels.shape[0] / batch_size)

with tf.Session(graph=graph_1_layer_sgd) as session:
  tf.global_variables_initializer().run()
  print("Initialized\n\n")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction],
                                    feed_dict=feed_dict)
    if (step % int(num_steps/10) == 0):
      print("step %s / %s" %(step, num_steps))
      print("Minibatch loss: %f" % l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("\n\n")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


step 0 / 1562
Minibatch loss: 16098.760742
Minibatch accuracy: 6.2%
Validation accuracy: 27.4%



step 156 / 1562
Minibatch loss: 6.897063
Minibatch accuracy: 81.2%
Validation accuracy: 77.6%



step 312 / 1562
Minibatch loss: 1.296801
Minibatch accuracy: 72.7%
Validation accuracy: 78.9%



step 468 / 1562
Minibatch loss: 1.184422
Minibatch accuracy: 78.1%
Validation accuracy: 78.7%



step 624 / 1562
Minibatch loss: 1.120281
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%



step 780 / 1562
Minibatch loss: 1.075450
Minibatch accuracy: 80.5%
Validation accuracy: 78.8%



step 936 / 1562
Minibatch loss: 1.145546
Minibatch accuracy: 79.7%
Validation accuracy: 80.7%



step 1092 / 1562
Minibatch loss: 0.984700
Minibatch accuracy: 85.2%
Validation accuracy: 78.0%



step 1248 / 1562
Minibatch loss: 1.098441
Minibatch accuracy: 82.0%
Validation accuracy: 79.4%



step 1404 / 1562
Minibatch loss: 1.151491
Minibatch accuracy: 76.6%
Validation accuracy: 80.0%



step 1560 / 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [45]:
# let's take a fraction only of the train set
fraction = 1 / 500
train_dataset_sub_set = train_dataset[:int(train_dataset.shape[0]*fraction),:]
train_labels_sub_set = train_labels[:int(train_labels.shape[0]*fraction),:]

train_labels_sub_set.shape, train_dataset_sub_set.shape # , train_labels

((400, 10), (400, 784))

In [46]:
batch_size = 128
num_steps = int(train_labels_sub_set.shape[0] / batch_size)
num_steps

3

In [47]:
with tf.Session(graph=graph_1_layer_sgd) as session:
  tf.global_variables_initializer().run()
  print("Initialized\n\n")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_sub_set.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_sub_set[offset:(offset + batch_size), :]
    batch_labels = train_labels_sub_set[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction],
                                    feed_dict=feed_dict)
    if (step % int(num_steps/np.min([num_steps, 10])) == 0):
      print("step %s / %s" %(step, num_steps))
      print("Minibatch loss: %f" % l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(),
                                                     valid_labels))
      print("\n\n")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


step 0 / 3
Minibatch loss: 16132.888672
Minibatch accuracy: 7.0%
Validation accuracy: 30.3%



step 1 / 3
Minibatch loss: 16076.161133
Minibatch accuracy: 31.2%
Validation accuracy: 32.7%



step 2 / 3
Minibatch loss: 15136.698242
Minibatch accuracy: 22.7%
Validation accuracy: 53.5%



Test accuracy: 57.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [49]:
num_hidden_layers = 1024
beta_l2_1_layer_sgd = 0.1
drop_out_ratio = 0.5


batch_size = 128

graph_1_layer_sgd_with_dropout = tf.Graph()
with graph_1_layer_sgd_with_dropout.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size,
                                             num_hidden_layers]))
  biases = tf.Variable(tf.zeros([num_hidden_layers]))
  preactivations_hidden = tf.add(tf.matmul(tf_train_dataset, weights), biases)

  # Hidden layer.
  activations_hidden = tf.nn.relu(preactivations_hidden)
  weights_hidden = tf.Variable(tf.truncated_normal([num_hidden_layers, num_labels]))
  bias_hidden = tf.Variable(tf.random_normal([num_labels]))
  logit_hidden = tf.nn.dropout(tf.add(tf.matmul(activations_hidden, weights_hidden),
                                      bias_hidden),
                               keep_prob=drop_out_ratio)
    
    
  # Training computation.
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,
                                                                logits=logit_hidden))
          + 0.5 * tf.nn.l2_loss(weights) * beta_l2_1_layer_sgd
          + 0.5 * tf.nn.l2_loss(biases) * beta_l2_1_layer_sgd
          + 0.5 * tf.nn.l2_loss(weights_hidden) * beta_l2_1_layer_sgd
          + 0.5 * tf.nn.l2_loss(bias_hidden) * beta_l2_1_layer_sgd
         )


  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logit_hidden)
  valid_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset,
                                                                                   weights),
                                                                         biases)),
                                                    weights_hidden),
                                          bias_hidden))
  test_prediction = tf.nn.softmax(tf.add(tf.matmul(tf.nn.relu(tf.add(tf.matmul(tf_test_dataset,
                                                                                  weights),
                                                                         biases)),
                                                    weights_hidden),
                                          bias_hidden))

In [50]:
num_steps = 3001

with tf.Session(graph=graph_1_layer_sgd_with_dropout) as session:
  tf.global_variables_initializer().run()
  print("Initialized\n\n")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction],
                                    feed_dict=feed_dict)
    if (step % int(num_steps/np.min([num_steps, 10])) == 0):
      print("step %s / %s" %(step, num_steps))
      print("Minibatch loss: %f" % l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(),
                                                     valid_labels))
      print("\n\n")
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized


step 0 / 3001
Minibatch loss: 16342.564453
Minibatch accuracy: 10.9%
Validation accuracy: 27.7%



step 300 / 3001
Minibatch loss: 1.626547
Minibatch accuracy: 53.9%
Validation accuracy: 76.5%



step 600 / 3001
Minibatch loss: 1.759867
Minibatch accuracy: 44.5%
Validation accuracy: 78.4%



step 900 / 3001
Minibatch loss: 1.794249
Minibatch accuracy: 43.0%
Validation accuracy: 77.2%



step 1200 / 3001
Minibatch loss: 1.607568
Minibatch accuracy: 49.2%
Validation accuracy: 77.0%



step 1500 / 3001
Minibatch loss: 1.833777
Minibatch accuracy: 39.1%
Validation accuracy: 77.3%



step 1800 / 3001
Minibatch loss: 1.609981
Minibatch accuracy: 51.6%
Validation accuracy: 79.1%



step 2100 / 3001
Minibatch loss: 1.732864
Minibatch accuracy: 43.8%
Validation accuracy: 78.5%



step 2400 / 3001
Minibatch loss: 1.748833
Minibatch accuracy: 46.1%
Validation accuracy: 76.2%



step 2700 / 3001
Minibatch loss: 1.564802
Minibatch accuracy: 51.6%
Validation accuracy: 77.3%



step 300

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


## -> TO DO !